In [3]:
import json 
import numpy as np 
import torch
from tqdm.notebook import tqdm 

from fugw_solver.solver import solver

# from collections import defaultdict
# data = defaultdict(list)
# with open("dataset/full.jsonl", 'r') as file:
#     for line in file:
#         j = json.loads(line)
#         for k,v in j.items():
#             data[k].append(v)

# with open("dataset/mutag.json", 'w') as f:
#     json.dump(data, f)

with open("dataset/mutag.json", 'r') as f:
    data = json.load(f)

In [4]:
### Hyperparameter
k = 10

In [5]:
features = data["node_feat"]
n = len(features)
y = data["y"]
adjacencies = []
for (sources, targets), num_nodes in zip(data["edge_index"], data["num_nodes"]):
    adj = np.zeros((num_nodes, num_nodes))
    adj[sources, targets] = 1
    adjacencies.append(adj)

In [8]:
# Test
source_features = torch.tensor(features[0]).T.float()
target_features = torch.tensor(features[1]).T.float()
source_geometry = torch.tensor(adjacencies[0]).float()
target_geometry = torch.tensor(adjacencies[1]).float()
P, _ = solver(source_features, target_features, source_geometry, target_geometry)
P.sum()

tensor(0.9662)

In [11]:
def predict(i : int, k:int, rho):
    """Predicts the class of the i-th molecule using a k-NN"""
    source_features = torch.tensor(features[i]).T.float()
    source_geometry = torch.tensor(adjacencies[i]).float()
    distances = []
    for j, (target_features, target_geometry, classif) in enumerate(zip(features, adjacencies, y)):
        if j == i:
            continue
        target_features = torch.tensor(target_features).T.float()
        target_geometry = torch.tensor(target_geometry).float()
        P, loss_ls = solver(source_features, target_features, source_geometry, target_geometry, rho=rho)
        distances.append((loss_ls[-1]["total"], classif))

    distances.sort(key=lambda x: x[0])[:k]
    classes = list(map(lambda x: x[1], distances))

    return 1 if classes.sum() >= classes.len() / 2 else 0

In [ ]:
accuracy = 0
rho = 1
for i in tqdm(range(n)):
    prediction = predict(i, k, rho)
    accuracy += (predict == y[i])
accuracy / n

  0%|          | 0/188 [00:00<?, ?it/s]

In [ ]:
accuracy = 0
rho = 100
for i in tqdm(range(n)):
    prediction = predict(i, k, rho)
    accuracy += (predict == y[i])
accuracy / n